In [2]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), './'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading  Morning Peak

Load original dataset and preprocess the loaded data.

In [ ]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('./etth_mp/ori_etth_mp24.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('./etth_mp/generate_etth_mp24.npy')
 
print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

b,t,n = fake_data.shape
# ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)
# fake_data = fake_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# ori_data = ori_data[:fake_data.shape[0]*fake_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (2881, 24, 7)
fake shape is:  (2881, 24, 7)


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed







## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [17]:
for j in range(3):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 :  mp  10mile  0.292   0.309
# seed 12345 :  ep  10mile  0.151


# uncondi   2.3  



Iter 0:  context-fid = 1.6986284536257041 

Iter 1:  context-fid = 2.2373361566673706 

Iter 2:  context-fid = 2.3729883231429136 

Iter 3:  context-fid = 2.0512278936608124 

Iter 4:  context-fid = 1.8356401974279053 

Final Score:  2.0391642049049414 ± 0.34468356449967946
Iter 0:  context-fid = 2.072091546076176 

Iter 1:  context-fid = 1.3325190142330492 

Iter 2:  context-fid = 2.0348614532719984 

Iter 3:  context-fid = 2.0578781936449184 

Iter 4:  context-fid = 2.015397234938828 

Final Score:  1.902549488432994 ± 0.3965786793717447
Iter 0:  context-fid = 2.040920051533032 

Iter 1:  context-fid = 0.8115768304522557 

Iter 2:  context-fid = 0.8772127509612841 

Iter 3:  context-fid = 2.5617460359990387 

Iter 4:  context-fid = 1.3817423317699058 

Final Score:  1.5346396001431033 ± 0.9394226088265512


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import warnings
warnings.filterwarnings("ignore")


import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

## Data Loading Morning Peak

Load original dataset and preprocess the loaded data.

In [8]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('./etth_mp/ori_etth_mp24.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('./etth_mp/generate_etth_mp24.npy')
print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)
b,t,n = ori_data.shape


ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)
fake_data = fake_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# fake_data = fake_data[:ori_data.shape[0]*ori_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (2880, 24, 7)
fake shape is:  (2880, 24, 7)
ori shape is:  (20160, 24, 1)
fake shape is:  (20160, 24, 1)


## Evaluate the generated data

### 1. Discriminative score

To evaluate the classification accuracy between original and synthetic data using post-hoc RNN network. The output is | classification accuracy - 0.5 |.

- metric_iteration: the number of iterations for metric computation.

In [ ]:
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:ori_data.shape[0]])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('etth:')
display_scores(discriminative_score)
print()
# seed 12345  Final Score:  0.0731896551724138 ± 0.005795392020461213
# seed 2024  Final Score:  0.06945402298850574 ± 0.010002021083688875

# univariate Final Score:  0.004115022310361938 ± 0.003740431320553614


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 2000/2000 [01:01<00:00, 32.62it/s]


Iter 0:  0.017026578073089715 , 0.48518826135105203 , 0.5488648947951273 



training: 100%|██████████| 2000/2000 [00:58<00:00, 34.22it/s]


Iter 1:  0.00567552602436322 , 0.5170957918050941 , 0.4715531561461794 



...

Iter 2:  0.012423864894795134 , 0.46047895902547065 , 0.5146733111849391 



training: 100%|██████████| 2000/2000 [00:56<00:00, 35.60it/s]


Iter 3:  0.007405869324473957 , 0.6319905869324474 , 0.38282115171650055 



training: 100%|██████████| 2000/2000 [00:52<00:00, 37.97it/s]


Iter 4:  0.015053986710963474 , 0.5393133997785161 , 0.49079457364341084 

etth:
Final Score:  0.0115171650055371 ± 0.0060419535687656544



## Evaluate the generated data

### 2. Predictive score

To evaluate the prediction performance on train on synthetic, test on real setting. More specifically, we use Post-hoc RNN architecture to predict one-step ahead and report the performance in terms of MAE. 

The model learns to predict the last dimension with one more step.

In [ ]:
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:ori_data.shape[0]])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

# univariate Final Score:  0.1605687830457955 ± 3.0256014600636085e-05

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 5000/5000 [01:50<00:00, 45.41it/s]


0  epoch:  0.2036337936773948 



training: 100%|██████████| 5000/5000 [01:48<00:00, 46.13it/s]


1  epoch:  0.20367806120860535 



...

2  epoch:  0.2037564074033652 



training: 100%|██████████| 5000/5000 [01:48<00:00, 45.93it/s]


3  epoch:  0.20367786735220364 



training: 100%|██████████| 5000/5000 [01:52<00:00, 44.52it/s]


4  epoch:  0.20359909053625458 

sine:
Final Score:  0.2036690440355647 ± 7.326764080408571e-05

